In [17]:
import pandas as pd
import boto3
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine

In [18]:
load_dotenv()

ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
HOSTNAME = os.getenv('HOSTNAME')
USERNAME = os.getenv('USERNAME')
PASSWORD = os.getenv('PASSWORD')
TABLE_NAME = os.getenv('TABLE_NAME')
DATABASE_NAME = os.getenv('DATABASE_NAME')
BUCKET_NAME = os.getenv('BUCKET_NAME')

In [5]:
#get data
session = boto3.Session(
    aws_access_key_id=ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
)

df1 = pd.read_csv(f's3://{BUCKET_NAME}/bengals.csv')
df2 = pd.read_csv(f's3://{BUCKET_NAME}/boyd_receiving.csv')
df3 = pd.read_csv(f's3://{BUCKET_NAME}/chase_receiving.csv')
df4 = pd.read_csv(f's3://{BUCKET_NAME}/higgins_receiving.csv')

In [19]:
#transform data
df2.columns = ['Week', 'Boyd Yards', 'Boyd TD']
df3.columns = ['Week', 'Chase Yards', 'Chase TD']
df4.columns = ['Week', 'Higgins Yards', 'Higgins TD']
merge1 = pd.merge(df1, df2, on='Week')
merge2 = pd.merge(merge1, df3, on='Week')
final_merge = pd.merge(merge2, df4, on='Week')
final_merge['Result'] = final_merge['Result'].map({1.0: 'Win', 0: 'Loss'})

In [38]:
#load to sql server
engine = create_engine(f'postgresql://{USERNAME}:{PASSWORD}@{HOSTNAME}:5432/{DATABASE_NAME}')
final_merge.to_sql('public.jordan_tang', con=engine, if_exists='replace')
# final_merge.to_sql('jordan_tang', con=engine)
with engine.connect() as conn:
    result = conn.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")
    for row in result:
        print(row)

NoSuchTableError: jordan_tang

In [20]:
print(df1.head())
print(df2.head())
print(df3.head())
print(df4.head())
print(final_merge.head())

   Week Opponent Location  Result
0  PRE1       TB     Away     1.0
1  PRE2      WSH     Away     0.0
2  PRE3      MIA     Home     0.0
3  REG1      MIN     Home     1.0
4  REG2      CHI     Away     0.0
   Week  Boyd Yards  Boyd TD
0  REG1          32        0
1  REG2          73        0
2  REG3          36        1
3  REG4         118        0
4  REG5          24        0
   Week  Chase Yards  Chase TD
0  REG1          101         1
1  REG2           54         1
2  REG3           65         2
3  REG4           77         0
4  REG5          159         1
   Week  Higgins Yards  Higgins TD
0  REG1             58           1
1  REG2             60           1
2  REG5             32           0
3  REG6             44           0
4  REG7             62           0
   Week Opponent Location Result  Boyd Yards  Boyd TD  Chase Yards  Chase TD  \
0  REG1      MIN     Home    Win          32        0          101         1   
1  REG2      CHI     Away   Loss          73        0           54